In [1]:
# changing in order to not reset paths
%cd -q ../../project/parallel_synthesis/experiments 

from copy import copy
from protera_stability.config.lazy import LazyCall as L
from protera_stability.config.instantiate import instantiate
from protera_stability.config.common.optim import Adam, AdamW, SGD

In [2]:
from protera_stability.config.common import mlp_esm

## Base optimizers

In [4]:
adam = copy(Adam)
model = instantiate(mlp_esm)
adam.params.model = model
optim =  instantiate(adam)

In [11]:
list(optim.param_groups[0]["params"])[0]

Parameter containing:
tensor([[ 0.0194,  0.0091, -0.0241,  ...,  0.0177,  0.0169,  0.0245],
        [-0.0027,  0.0119,  0.0034,  ...,  0.0191,  0.0264,  0.0105],
        [-0.0176,  0.0035, -0.0072,  ..., -0.0001,  0.0007, -0.0182],
        ...,
        [-0.0176, -0.0086, -0.0093,  ...,  0.0204,  0.0079, -0.0016],
        [ 0.0083,  0.0067,  0.0200,  ...,  0.0203,  0.0247,  0.0105],
        [-0.0131,  0.0201, -0.0013,  ...,  0.0006, -0.0142, -0.0206]],
       requires_grad=True)

In [7]:
import torch

X = torch.randn((1, 1280))
y = torch.Tensor([[0.5]])

In [10]:
model.layers[0].weight

Parameter containing:
tensor([[ 0.0194,  0.0091, -0.0241,  ...,  0.0177,  0.0169,  0.0245],
        [-0.0027,  0.0119,  0.0034,  ...,  0.0191,  0.0264,  0.0105],
        [-0.0176,  0.0035, -0.0072,  ..., -0.0001,  0.0007, -0.0182],
        ...,
        [-0.0176, -0.0086, -0.0093,  ...,  0.0204,  0.0079, -0.0016],
        [ 0.0083,  0.0067,  0.0200,  ...,  0.0203,  0.0247,  0.0105],
        [-0.0131,  0.0201, -0.0013,  ...,  0.0006, -0.0142, -0.0206]],
       requires_grad=True)

In [9]:
criterion = torch.nn.MSELoss()
y_hat = model(X)
print(y_hat)
loss = criterion(y_hat, y)
loss.backward()
optim.step()
optim.zero_grad()

tensor([[-0.1304]], grad_fn=<AddmmBackward>)


In [8]:
y_hat == model(X)

tensor([[False]])

In [4]:
adamw = copy(AdamW)
adamw.params.model = mlp_esm
instantiate(adamw)

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0.01
)

In [5]:
sgd = copy(SGD)
sgd.params.model = mlp_esm
instantiate(sgd)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.02
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001
)

## Base Schedulers

In [6]:
from protera_stability.config.common import StepLR, CosineLR

In [8]:
linear_sched = copy(StepLR)
linear_sched.optimizer = sgd
instantiate(linear_sched)

In [9]:
cosine_sched = copy(CosineLR)
cosine_sched.optimizer = sgd
instantiate(cosine_sched)